Scientific Computing II Numerical Methods for Engineers

Chapter 27 - Boundary-Value & Eigenvalue Equations

In [22]:
import nbimporter
import math
from CISC601_Ch25Examples import fourth_order, fourth_order_x_only
import numpy as np

## Example 27.1 The Shooting Method 
# Constants for the problem
L = 10   # Length in meters
Ta = 20  # Ambient temperature in degrees Celsius
hp = 0.01  # Heat transfer coefficient
T1 = 40  # Temperature at x=0
T2 = 200  # Boundary condition at x=L
h = 2    # Step size for x

# ODE system
def dy1(x, y):
    return y[1]  # dy1/dx = z

def dy2(x, y):
    return hp * (y[0] - Ta)  # dz/dx = hp * (T - Ta)

def dyn(x, y):
    return np.array([dy1(x, y), dy2(x, y)])

# Fourth-order Runge-Kutta integration (using your provided function)
def rk4_step(x, y, h):
    return fourth_order(x, y, h, dyn)

# Boundary shooting function
def shoot_for_z0(z0, x_range):
    y = np.array([T1, z0])  # Initial conditions [T(0), z(0)]
    results = []
    
    for x in x_range:
        results.append(y[0])  # Store temperature T
        y = rk4_step(x, y, h)  # Perform a Runge-Kutta step
    return np.array(results)

# Main shooting method process
x_range = np.arange(0, L + h, h)

# Try shooting with z(0) = 10
z0_1 = 10
print(f'Shooting for z(0) = {z0_1}')
y_shot1 = shoot_for_z0(z0_1, x_range)

# Try shooting with z(0) = 20
z0_2 = 20
print(f'Shooting for z(0) = {z0_2}')
y_shot2 = shoot_for_z0(z0_2, x_range)

# Linear interpolation to estimate z(0) for T(L) = 200 
## (Equation 18.2)  -- But this doesn't work because we have specifica values, not a function
def linear_interp(f, x0,x1 ,x):  
    top = (f(x1) - f(x0) * (x-x0))
    bottom = x1 - x0
    return f(x0) + top/bottom 

def linear_interp_REFACTOR(fx0, fx1, x0,x1,x):
    top = (fx1-fx0)*(x-x0)
    bottom = x1 - x0
    return fx0 + top/bottom

z0_interp = linear_interp_REFACTOR(z0_1,z0_2, y_shot1[-1], y_shot2[-1], T2)
print(f'Interpolated z(0) = {z0_interp}')

# Final shooting with interpolated z(0)
y_shot_interp = shoot_for_z0(z0_interp, x_range)

# Results
print(f"x points: {x_range}")
print(f"Results for z(0) = {z0_interp}: {y_shot_interp}")




Shooting for z(0) = 10
[10.   0.2] [10.2  0.3] [10.3    0.302]
[10.60333333  0.40534667] [11.00868  0.51138] [11.11471333  0.51543347]
[11.63220484  0.62696107] [12.25916591  0.74328312] [12.37548796  0.74955273]
[13.12790604  0.87373714] [14.00164318  1.0050162 ] [14.13292224  1.01375358]
[15.15046347  1.15557864] [16.30604211  1.30708328] [16.45754675  1.31863907]
[17.78104777  1.48379659] [19.26484436  1.66160706] [19.44265484  1.67644503]
Shooting for z(0) = 20
[20.   0.2] [20.2  0.4] [20.4    0.402]
[20.804    0.60668] [21.41068  0.81472] [21.61872    0.8207868]
[22.442916    1.03770791] [23.48062391  1.26213707] [23.70505307  1.27251415]
[24.98252244  1.51038196] [26.4929044   1.76020718] [26.74272963  1.775311  ]
[28.52474074  2.04367174] [30.56841248  2.32891915] [30.85365989  2.34935587]
[33.21172965  2.65897954] [35.87070918  2.99109683] [36.20282648  3.01768663]
Interpolated z(0) = 12.69067393711733
[12.69067394  0.2       ] [12.89067394  0.32690674] [13.01758068  0.32890674

In [27]:
## Example 27.3 Finite Difference Approximation of Boundary-Value Problems

import numpy as np

# Constants for the problem

n = int(L / h) + 1  # Number of points including boundaries

# Set up the grid
x = np.linspace(0, L, n)

# Create the coefficient matrix A and the right-hand side vector b
A = np.zeros((n, n))
b = np.zeros(n)

# Boundary conditions
A[0, 0] = 1  # T(0) = T1
b[0] = T1

A[-1, -1] = 1  # T(L) = T2
b[-1] = T2

# Finite difference method for internal points (1 to n-2)
for i in range(1, n-1):
    A[i, i-1] = 1       # T_{i-1}
    A[i, i] = -2 - hp * h**2  # T_i (adjusted for the hp term)
    A[i, i+1] = 1       # T_{i+1}
    b[i] = -hp * h**2 * Ta  # Right-hand side for each internal point

# Solve the linear system
T = np.linalg.solve(A, b)

# Output results
print(f"x points: {x}")
print(f"T points: {T}")


x points: [ 0.  2.  4.  6.  8. 10.]
T points: [ 40.          65.96983437  93.77846211 124.53822833 159.47952369
 200.        ]


In [29]:
## Example 27.4 Eigen values and vectors for Mass-Spring System

import numpy as np

# Given constants
k = 200  # N/m (spring constant)
m1 = 40  # kg (mass 1)
m2 = 40  # kg (mass 2)

# Construct the matrix
M = np.array([[2 * k / m1, -k / m1],
              [-k / m2, 2 * k / m2]])

# Compute the eigenvalues (which are ω^2)
eigenvalues, eigenvectors = np.linalg.eig(M)

# The eigenvalues are ω^2, so to get ω, take the square root
frequencies = np.sqrt(eigenvalues)

# Output the eigenvalues (ω^2) and natural frequencies (ω)
print(f"Eigenvalues (ω^2): {eigenvalues}")
print(f"Natural frequencies (ω): {frequencies}")


Eigenvalues (ω^2): [15.  5.]
Natural frequencies (ω): [3.87298335 2.23606798]
